# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [3]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # TODO: Implement Function
    source_sentences = source_text.split("\n")
    source_id_text = list()
    for sentence in source_sentences:
        source_id_word = [source_vocab_to_int[word] for word in sentence.split()]
        source_id_text.append(source_id_word)
        
        
    target_sentences = target_text.split("\n")
    target_id_text = list()
    for sentence in target_sentences:
        target_id_word = [target_vocab_to_int[word] for word in sentence.split()]
        target_id_word.append(target_vocab_to_int['<EOS>'])
        target_id_text.append(target_id_word)
    
    
    return source_id_text, target_id_text

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0


/Users/nbhad/anaconda/envs/language-translation/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: No GPU found. Please use a GPU to train your neural network.
  from ipykernel import kernelapp as app


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [7]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    # TODO: Implement Function
    input_data = tf.placeholder(tf.int32, [None, None] , name="input")
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    learning_rate = tf.placeholder(tf.float32)
    keep_probability = tf.placeholder(tf.float32, name="keep_prob")
    target_sequence_length = tf.placeholder(tf.int32, [None] , name="target_sequence_length")
    max_target_sequence_length = tf.reduce_max(target_sequence_length, name='max_target_len')
    source_sequence_length = tf.placeholder(tf.int32, [None] , name="source_sequence_length")
    return input_data, targets, learning_rate, keep_probability, target_sequence_length, max_target_sequence_length, source_sequence_length


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [8]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # TODO: Implement Function
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    dec_input = tf.concat([tf.fill([batch_size, 1], target_vocab_to_int['<GO>']), ending], 1)
    return dec_input

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [9]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    # TODO: Implement Function
    #Encoder embedding
    enc_embed_input = tf.contrib.layers.embed_sequence(rnn_inputs, source_vocab_size, encoding_embedding_size)
    
    # RNN cell
    def make_cell(rnn_size):
        enc_cell = tf.contrib.rnn.LSTMCell(rnn_size, initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        return enc_cell
    
    enc_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])
    enc_output, enc_state = tf.nn.dynamic_rnn(enc_cell, enc_embed_input, sequence_length=source_sequence_length, dtype=tf.float32)
    
    return enc_output, enc_state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [10]:

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    # TODO: Implement Function
    training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,
                                                       sequence_length=target_sequence_length,
                                                        time_major=False)
    training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                      training_helper,
                                                      encoder_state,
                                                      output_layer)
    training_decoder_output, _ = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                                  impute_finished=True,
                                                                  maximum_iterations=max_summary_length)
    
    return training_decoder_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [11]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    # TODO: Implement Function
    
    start_tokens = tf.tile(tf.constant([start_of_sequence_id], dtype=tf.int32), [batch_size], name='start_tokens')
    
    inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings,
                                                               start_tokens,
                                                               end_of_sequence_id)
    
    inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                       inference_helper,
                                                       encoder_state,
                                                       output_layer)
    
    inference_decoder_output, _ = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                                                   impute_finished=True,
                                                                   maximum_iterations=max_target_sequence_length)
    
    
    return inference_decoder_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [12]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    
    #Embed the target sequences
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)
    
    #Construct the decoder LSTM cell
    def make_cell(rnn_size):
        dec_cell = tf.contrib.rnn.LSTMCell(rnn_size,
                                          initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        return dec_cell
    
    dec_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])
    
    #Create an output layer to map the outputs of the decoder to the elements of our vocabulary
    output_layer = Dense(target_vocab_size,
                        kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
    
    #Training Logits
    with tf.variable_scope("decode"):
        training_decoder_output = decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)
    
    #Inference Logits.
    with tf.variable_scope("decode", reuse=True):
        start_of_sequence_id = target_vocab_to_int['<GO>']
        end_of_sequence_id = target_vocab_to_int['<EOS>']
        inference_decoder_output = decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, target_vocab_size, output_layer, batch_size, keep_prob)
    
    return training_decoder_output, inference_decoder_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Apply embedding to the input data for the encoder.
- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Apply embedding to the target data for the decoder.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [13]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    
    
    
    #Encode the input.
    _, enc_state = encoding_layer(input_data, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, enc_embedding_size)
    
    #Process target data
    dec_input = process_decoder_input(target_data, target_vocab_to_int, batch_size)
    
    
    #Decode the encoded input
    training_decoder_output, inference_decoder_output = decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)
                                  
    
    return training_decoder_output, inference_decoder_output


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [14]:
# Number of Epochs
epochs = 60
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 50
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 15
decoding_embedding_size = 15
# Learning Rate
learning_rate = 0.001
# Dropout Keep Probability
keep_probability = 0.5
display_step = 10

### Build the Graph
Build the graph using the neural network you implemented.

In [15]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [16]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch   10/1077 - Train Accuracy: 0.1982, Validation Accuracy: 0.3050, Loss: 5.5987
Epoch   0 Batch   20/1077 - Train Accuracy: 0.2316, Validation Accuracy: 0.3050, Loss: 4.7091
Epoch   0 Batch   30/1077 - Train Accuracy: 0.2277, Validation Accuracy: 0.3050, Loss: 4.1407
Epoch   0 Batch   40/1077 - Train Accuracy: 0.2316, Validation Accuracy: 0.3050, Loss: 3.8330
Epoch   0 Batch   50/1077 - Train Accuracy: 0.2199, Validation Accuracy: 0.3050, Loss: 3.6796
Epoch   0 Batch   60/1077 - Train Accuracy: 0.2972, Validation Accuracy: 0.3363, Loss: 3.4110
Epoch   0 Batch   70/1077 - Train Accuracy: 0.2307, Validation Accuracy: 0.3366, Loss: 3.5934
Epoch   0 Batch   80/1077 - Train Accuracy: 0.2719, Validation Accuracy: 0.3366, Loss: 3.3541
Epoch   0 Batch   90/1077 - Train Accuracy: 0.2668, Validation Accuracy: 0.3370, Loss: 3.3258
Epoch   0 Batch  100/1077 - Train Accuracy: 0.2617, Validation Accuracy: 0.3370, Loss: 3.2771
Epoch   0 Batch  110/1077 - Train Accuracy: 0.2883, Validati

Epoch   0 Batch  890/1077 - Train Accuracy: 0.5219, Validation Accuracy: 0.5050, Loss: 1.5859
Epoch   0 Batch  900/1077 - Train Accuracy: 0.4449, Validation Accuracy: 0.5036, Loss: 1.7918
Epoch   0 Batch  910/1077 - Train Accuracy: 0.4762, Validation Accuracy: 0.5025, Loss: 1.6491
Epoch   0 Batch  920/1077 - Train Accuracy: 0.4578, Validation Accuracy: 0.5135, Loss: 1.7029
Epoch   0 Batch  930/1077 - Train Accuracy: 0.4594, Validation Accuracy: 0.5103, Loss: 1.6609
Epoch   0 Batch  940/1077 - Train Accuracy: 0.4523, Validation Accuracy: 0.5071, Loss: 1.7109
Epoch   0 Batch  950/1077 - Train Accuracy: 0.4468, Validation Accuracy: 0.4812, Loss: 1.6353
Epoch   0 Batch  960/1077 - Train Accuracy: 0.4554, Validation Accuracy: 0.4730, Loss: 1.5531
Epoch   0 Batch  970/1077 - Train Accuracy: 0.4391, Validation Accuracy: 0.4936, Loss: 1.6632
Epoch   0 Batch  980/1077 - Train Accuracy: 0.4734, Validation Accuracy: 0.4933, Loss: 1.5698
Epoch   0 Batch  990/1077 - Train Accuracy: 0.4124, Validati

Epoch   1 Batch  700/1077 - Train Accuracy: 0.4766, Validation Accuracy: 0.5160, Loss: 0.9700
Epoch   1 Batch  710/1077 - Train Accuracy: 0.4723, Validation Accuracy: 0.5181, Loss: 0.9938
Epoch   1 Batch  720/1077 - Train Accuracy: 0.4655, Validation Accuracy: 0.5213, Loss: 1.0729
Epoch   1 Batch  730/1077 - Train Accuracy: 0.4883, Validation Accuracy: 0.5149, Loss: 0.9904
Epoch   1 Batch  740/1077 - Train Accuracy: 0.5039, Validation Accuracy: 0.5334, Loss: 0.9787
Epoch   1 Batch  750/1077 - Train Accuracy: 0.4898, Validation Accuracy: 0.5252, Loss: 0.9659
Epoch   1 Batch  760/1077 - Train Accuracy: 0.4871, Validation Accuracy: 0.5391, Loss: 1.0088
Epoch   1 Batch  770/1077 - Train Accuracy: 0.5082, Validation Accuracy: 0.5224, Loss: 0.9432
Epoch   1 Batch  780/1077 - Train Accuracy: 0.4789, Validation Accuracy: 0.5213, Loss: 1.0215
Epoch   1 Batch  790/1077 - Train Accuracy: 0.4266, Validation Accuracy: 0.5185, Loss: 1.0327
Epoch   1 Batch  800/1077 - Train Accuracy: 0.4492, Validati

Epoch   2 Batch  510/1077 - Train Accuracy: 0.5484, Validation Accuracy: 0.5675, Loss: 0.7239
Epoch   2 Batch  520/1077 - Train Accuracy: 0.5878, Validation Accuracy: 0.5842, Loss: 0.7018
Epoch   2 Batch  530/1077 - Train Accuracy: 0.5312, Validation Accuracy: 0.5700, Loss: 0.7454
Epoch   2 Batch  540/1077 - Train Accuracy: 0.5285, Validation Accuracy: 0.5707, Loss: 0.6967
Epoch   2 Batch  550/1077 - Train Accuracy: 0.5238, Validation Accuracy: 0.5714, Loss: 0.7687
Epoch   2 Batch  560/1077 - Train Accuracy: 0.5387, Validation Accuracy: 0.5849, Loss: 0.7072
Epoch   2 Batch  570/1077 - Train Accuracy: 0.5329, Validation Accuracy: 0.5806, Loss: 0.7480
Epoch   2 Batch  580/1077 - Train Accuracy: 0.5592, Validation Accuracy: 0.5792, Loss: 0.6762
Epoch   2 Batch  590/1077 - Train Accuracy: 0.5362, Validation Accuracy: 0.5604, Loss: 0.7471
Epoch   2 Batch  600/1077 - Train Accuracy: 0.5670, Validation Accuracy: 0.5732, Loss: 0.6931
Epoch   2 Batch  610/1077 - Train Accuracy: 0.5358, Validati

Epoch   3 Batch  320/1077 - Train Accuracy: 0.6160, Validation Accuracy: 0.5980, Loss: 0.6008
Epoch   3 Batch  330/1077 - Train Accuracy: 0.6441, Validation Accuracy: 0.5877, Loss: 0.6119
Epoch   3 Batch  340/1077 - Train Accuracy: 0.5259, Validation Accuracy: 0.5742, Loss: 0.6106
Epoch   3 Batch  350/1077 - Train Accuracy: 0.5559, Validation Accuracy: 0.5643, Loss: 0.6474
Epoch   3 Batch  360/1077 - Train Accuracy: 0.5824, Validation Accuracy: 0.5852, Loss: 0.6043
Epoch   3 Batch  370/1077 - Train Accuracy: 0.5796, Validation Accuracy: 0.5906, Loss: 0.5746
Epoch   3 Batch  380/1077 - Train Accuracy: 0.6234, Validation Accuracy: 0.6048, Loss: 0.5734
Epoch   3 Batch  390/1077 - Train Accuracy: 0.5414, Validation Accuracy: 0.5977, Loss: 0.6077
Epoch   3 Batch  400/1077 - Train Accuracy: 0.5965, Validation Accuracy: 0.5881, Loss: 0.5943
Epoch   3 Batch  410/1077 - Train Accuracy: 0.5999, Validation Accuracy: 0.5962, Loss: 0.6048
Epoch   3 Batch  420/1077 - Train Accuracy: 0.6320, Validati

...

Epoch  55 Batch 1000/1077 - Train Accuracy: 0.9766, Validation Accuracy: 0.9730, Loss: 0.0091
Epoch  55 Batch 1010/1077 - Train Accuracy: 0.9797, Validation Accuracy: 0.9727, Loss: 0.0095
Epoch  55 Batch 1020/1077 - Train Accuracy: 0.9895, Validation Accuracy: 0.9620, Loss: 0.0077
Epoch  55 Batch 1030/1077 - Train Accuracy: 0.9859, Validation Accuracy: 0.9602, Loss: 0.0097
Epoch  55 Batch 1040/1077 - Train Accuracy: 0.9815, Validation Accuracy: 0.9670, Loss: 0.0094
Epoch  55 Batch 1050/1077 - Train Accuracy: 0.9977, Validation Accuracy: 0.9734, Loss: 0.0068
Epoch  55 Batch 1060/1077 - Train Accuracy: 0.9785, Validation Accuracy: 0.9599, Loss: 0.0098
Epoch  55 Batch 1070/1077 - Train Accuracy: 0.9742, Validation Accuracy: 0.9712, Loss: 0.0073
Epoch  56 Batch   10/1077 - Train Accuracy: 0.9667, Validation Accuracy: 0.9563, Loss: 0.0115
Epoch  56 Batch   20/1077 - Train Accuracy: 0.9766, Validation Accuracy: 0.9645, Loss: 0.0077
Epoch  56 Batch   30/1077 - Train Accuracy: 0.9832, Validati

Epoch  56 Batch  810/1077 - Train Accuracy: 0.9740, Validation Accuracy: 0.9652, Loss: 0.0087
Epoch  56 Batch  820/1077 - Train Accuracy: 0.9961, Validation Accuracy: 0.9606, Loss: 0.0084
Epoch  56 Batch  830/1077 - Train Accuracy: 0.9652, Validation Accuracy: 0.9627, Loss: 0.0139
Epoch  56 Batch  840/1077 - Train Accuracy: 0.9879, Validation Accuracy: 0.9641, Loss: 0.0117
Epoch  56 Batch  850/1077 - Train Accuracy: 0.9725, Validation Accuracy: 0.9634, Loss: 0.0157
Epoch  56 Batch  860/1077 - Train Accuracy: 0.9855, Validation Accuracy: 0.9638, Loss: 0.0096
Epoch  56 Batch  870/1077 - Train Accuracy: 0.9827, Validation Accuracy: 0.9656, Loss: 0.0088
Epoch  56 Batch  880/1077 - Train Accuracy: 0.9777, Validation Accuracy: 0.9673, Loss: 0.0093
Epoch  56 Batch  890/1077 - Train Accuracy: 0.9940, Validation Accuracy: 0.9556, Loss: 0.0096
Epoch  56 Batch  900/1077 - Train Accuracy: 0.9746, Validation Accuracy: 0.9659, Loss: 0.0139
Epoch  56 Batch  910/1077 - Train Accuracy: 0.9754, Validati

Epoch  57 Batch  620/1077 - Train Accuracy: 0.9754, Validation Accuracy: 0.9716, Loss: 0.0137
Epoch  57 Batch  630/1077 - Train Accuracy: 0.9891, Validation Accuracy: 0.9634, Loss: 0.0091
Epoch  57 Batch  640/1077 - Train Accuracy: 0.9799, Validation Accuracy: 0.9677, Loss: 0.0086
Epoch  57 Batch  650/1077 - Train Accuracy: 0.9805, Validation Accuracy: 0.9730, Loss: 0.0086
Epoch  57 Batch  660/1077 - Train Accuracy: 0.9824, Validation Accuracy: 0.9741, Loss: 0.0070
Epoch  57 Batch  670/1077 - Train Accuracy: 0.9826, Validation Accuracy: 0.9663, Loss: 0.0092
Epoch  57 Batch  680/1077 - Train Accuracy: 0.9598, Validation Accuracy: 0.9719, Loss: 0.0109
Epoch  57 Batch  690/1077 - Train Accuracy: 0.9523, Validation Accuracy: 0.9602, Loss: 0.0135
Epoch  57 Batch  700/1077 - Train Accuracy: 0.9855, Validation Accuracy: 0.9727, Loss: 0.0083
Epoch  57 Batch  710/1077 - Train Accuracy: 0.9738, Validation Accuracy: 0.9695, Loss: 0.0101
Epoch  57 Batch  720/1077 - Train Accuracy: 0.9794, Validati

Epoch  58 Batch  430/1077 - Train Accuracy: 0.9840, Validation Accuracy: 0.9549, Loss: 0.0098
Epoch  58 Batch  440/1077 - Train Accuracy: 0.9738, Validation Accuracy: 0.9680, Loss: 0.0121
Epoch  58 Batch  450/1077 - Train Accuracy: 0.9793, Validation Accuracy: 0.9545, Loss: 0.0107
Epoch  58 Batch  460/1077 - Train Accuracy: 0.9938, Validation Accuracy: 0.9620, Loss: 0.0090
Epoch  58 Batch  470/1077 - Train Accuracy: 0.9807, Validation Accuracy: 0.9588, Loss: 0.0134
Epoch  58 Batch  480/1077 - Train Accuracy: 0.9757, Validation Accuracy: 0.9549, Loss: 0.0103
Epoch  58 Batch  490/1077 - Train Accuracy: 0.9633, Validation Accuracy: 0.9563, Loss: 0.0122
Epoch  58 Batch  500/1077 - Train Accuracy: 0.9984, Validation Accuracy: 0.9698, Loss: 0.0062
Epoch  58 Batch  510/1077 - Train Accuracy: 0.9793, Validation Accuracy: 0.9567, Loss: 0.0099
Epoch  58 Batch  520/1077 - Train Accuracy: 0.9874, Validation Accuracy: 0.9709, Loss: 0.0074
Epoch  58 Batch  530/1077 - Train Accuracy: 0.9848, Validati

Epoch  59 Batch  240/1077 - Train Accuracy: 0.9941, Validation Accuracy: 0.9759, Loss: 0.0054
Epoch  59 Batch  250/1077 - Train Accuracy: 0.9680, Validation Accuracy: 0.9634, Loss: 0.0097
Epoch  59 Batch  260/1077 - Train Accuracy: 0.9743, Validation Accuracy: 0.9688, Loss: 0.0079
Epoch  59 Batch  270/1077 - Train Accuracy: 0.9855, Validation Accuracy: 0.9709, Loss: 0.0159
Epoch  59 Batch  280/1077 - Train Accuracy: 0.9691, Validation Accuracy: 0.9538, Loss: 0.0109
Epoch  59 Batch  290/1077 - Train Accuracy: 0.9809, Validation Accuracy: 0.9563, Loss: 0.0135
Epoch  59 Batch  300/1077 - Train Accuracy: 0.9934, Validation Accuracy: 0.9709, Loss: 0.0051
Epoch  59 Batch  310/1077 - Train Accuracy: 0.9738, Validation Accuracy: 0.9719, Loss: 0.0095
Epoch  59 Batch  320/1077 - Train Accuracy: 0.9875, Validation Accuracy: 0.9727, Loss: 0.0153
Epoch  59 Batch  330/1077 - Train Accuracy: 0.9820, Validation Accuracy: 0.9599, Loss: 0.0070
Epoch  59 Batch  340/1077 - Train Accuracy: 0.9836, Validati

### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [18]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [19]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [26]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # TODO: Implement Function
    sentence = sentence.lower()
    return [vocab_to_int.get(word,vocab_to_int['<UNK>']) for word in sentence.split()]


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [27]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [129, 175, 217, 210, 65, 114, 146]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [100, 167, 344, 10, 309, 64, 1]
  French Words: il gel fraises la pomme . <EOS>


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.